In [1]:
import sys
if ".." not in sys.path:
    sys.path.insert(0, "..") 

from src.utils import *

# Late fusion

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
audio_embs = torch.load('../embeddings/audio/raw.pt')
video_embs = torch.load('../embeddings/video/raw2.pt')

In [3]:
train_loader_audio, test_loader_audio = train_test_dataloader(audio_embs)

model_audio = MLP(input_dim = 768, hidden_dims = [512, 256], output_dim = 6).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_audio.parameters(), lr = 0.001)
num_epochs = 50

PARAMS_AUDIO = {'model': model_audio,
          'criterion': criterion,
          'optimizer': optimizer,
          'num_epochs': num_epochs,
          'train_loader': train_loader_audio,
          'test_loader': test_loader_audio,
          'device': DEVICE}


audio_outputs, audio_labels = train_late_fusion_model(**PARAMS_AUDIO)

/home/mweuro/venvy/best_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
train_loader_video, test_loader_video = train_test_dataloader(video_embs)

model_video = MLP(input_dim = 512, hidden_dims = [512, 256], output_dim = 6).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_video.parameters(), lr = 0.001)
num_epochs = 50

PARAMS_VIDEO = {'model': model_video,
          'criterion': criterion,
          'optimizer': optimizer,
          'num_epochs': num_epochs,
          'train_loader': train_loader_video,
          'test_loader': test_loader_video,
          'device': DEVICE}


video_outputs, video_labels = train_late_fusion_model(**PARAMS_VIDEO)

In [5]:
def late_fusion(audio_outputs, video_outputs, labels):
    idx = -1
    total_loss = 0
    all_fused_logits = []
    softmax = nn.Softmax(dim = 0)
    for i, j in zip(audio_outputs[idx], video_outputs[idx]):
        fused_logit = (softmax(i) + softmax(j)) / 2
        all_fused_logits.append(fused_logit)
    outputs = torch.stack(all_fused_logits, dim = 0)
    loss = criterion(outputs, torch.tensor(labels[idx]).to(DEVICE))
    total_loss += loss.item()
    preds = torch.argmax(outputs, dim = 1).cpu().numpy()
    accuracy = accuracy_score(labels[idx], preds)
    print(f"Val Loss: {total_loss:.4f}, Val Accuracy: {accuracy:.4f}")

In [6]:
late_fusion(audio_outputs, video_outputs, video_labels)

Val Loss: 1.6706, Val Accuracy: 0.3768
